In [1]:
import pandas as pd
import numpy as np
from oraDB import oraDB
conn,cur= oraDB.connect()

--- 0.07923150062561035econds ---
Connection successful!


In [5]:
def reformat_period(rpt_dt):
    import re
    return re.sub(r'(-)','_',rpt_dt)

def job_feature (rpt_dt):
    import pandas as pd
    import numpy as np
    import sys
    #from datetime import datetime
    import time
    from oraDB import oraDB
    from sqlalchemy import text
    #conn,cur= oraDB.connect()
   #now = datetime.now()
    #start_time = now.strftime("%H:%M:%S")
    rpt_dt = reformat_period(rpt_dt)
    start_time = time.time()
    dbEngine = oraDB.create_engine()
    with dbEngine.connect() as conn:
        #creating tables
        tbl_nm = 'CINS_FEATURE_STORE'
        #ALTER SESSION
        conn.execute(text("ALTER SESSION FORCE PARALLEL DML PARALLEL 64"))
        conn.execute(text("ALTER SESSION FORCE PARALLEL QUERY PARALLEL 64"))
        #drop tmp tables
        try:
            #drop = open('/opt/bitnami/jupyterhub-singleuser/Go Live/FTR_STORE/script_drop_tbl.txt','r') 
            drop = open('script_drop_tbl.txt','r') 
            query_drop = drop.read()
        except:
            print("Unable to read Drop Table text file")
            conn.close()
            sys.exit()
            drop.close()
        t = 0
        print('--------STARTING DROPING TABLE(S)--------')
        for sql_drop in query_drop.split(';'):

            sql_drop = sql_drop.replace("{RPT_DT}",rpt_dt)

            t = t + 1
            print(sql_drop)
            try:
                drop_time = time.time()
                conn.execute(text(sql_drop))
                conn.execute(text("COMMIT"))
                print('-----Finished dropping ' + str(t) + ' table(s)-----')
                print('Runtime: ', time.time() - drop_time)
            except:
                print("Unable to drop table(s)")
                conn.close()
                sys.exit()
        #CREATE TABLES
        try:
            #create_tbl = open('/opt/bitnami/jupyterhub-singleuser/Go Live/FTR_STORE/script_create_tbl.txt','r')
            create_tbl = open('script_create_tbl.txt','r')
            query_create = create_tbl.read()
        except:
            print("Unable to read Create Table text file")
            conn.close()
            sys.exit()
            create_tbl.close()
        n = 0
        print('--------STARTING CREATING TABLE(S)--------')
        for sql_create_tbl in query_create.split(';'):
            sql_create_tbl = sql_create_tbl.replace("{RPT_DT}",rpt_dt)
            n = n + 1
            print(sql_create_tbl)
            try:
                create_time = time.time()
                conn.execute(text(sql_create_tbl))
                conn.execute(text("COMMIT"))
                print('-----Finished creating ' + str(n) + ' table(s)-----')
                print('Runtime: ', time.time() - create_time)
            except:
                print("Unable to create table(s)")
                conn.close()
                sys.exit()
        #insert features
        try:
            #insert = open('/opt/bitnami/jupyterhub-singleuser/Go Live/FTR_STORE/script_insert_main.txt','r') 
            insert = open('script_insert_main.txt','r') 
            query_insert = insert.read()
        except:
            print("Unable to read Insert Feature text file")
            conn.close()
            sys.exit()    
            insert.close()
        m = 0
        print('--------STARTING INSERTING FEATURE(S)--------')
        
        
        #Delete 
        sql_del = f"""alter table {tbl_nm} truncate partition PERIOD_{rpt_dt}"""
        # sql_del = f"DELETE FROM {tbl_nm} WHERE RPT_DT = '{rpt_dt}'"
        try:
            print(sql_del)
            del_time = time.time()
            conn.execute(text(sql_del))
            print(f'-----Sucessfully deleted data in {tbl_nm} with report date = {rpt_dt}-----')
            print('Runtime: ', time.time() - del_time)
        except:
            print("Unable to delete feature(s)")
            conn.close()
            sys.exit()
            drop.close()
        #Insert
        for sql_insert in query_insert.split(';'):    
            sql_insert = sql_insert.replace("{RPT_DT}",rpt_dt) 
            sql_insert = sql_insert.replace("{TBL_NM}",tbl_nm)
            m = m + 1
            print(sql_insert)
            try:
                insert_time = time.time()
                insert_result = conn.execute(text(sql_insert))
                conn.execute(text("COMMIT"))
                print('-----Finished inserting ' + str(m) + ' feature(s)-----')
                print(f"""{insert_result.rowcount} rows inserted!""")
                print('Runtime: ', time.time() - insert_time)
            except Exception as e:
                print(e)
                print("Unable to insert feature(s)")
                conn.close()
                sys.exit()
        #drop tmp rpt_dt tables
        try:
            #drop = open('/opt/bitnami/jupyterhub-singleuser/Go Live/FTR_STORE/script_drop_tbl.txt','r') 
            drop_rpt_dt = open('script_drop_rpt_dt_tbl.txt','r') 
            query_drop_rpt_dt = drop_rpt_dt.read()
        except:
            print("Unable to read Drop Table text file")
            conn.close()
            sys.exit()
            drop_rpt_dt.close()
        o = 0
        print('--------STARTING DROPING REPORT DATE TABLE(S)--------')
        for sql_drop_rpt_dt in query_drop_rpt_dt.split(';'):

            sql_drop_rpt_dt = sql_drop_rpt_dt.replace("{RPT_DT}",rpt_dt)

            o = o + 1
            print(sql_drop_rpt_dt)
            try:
                drop_rpt_dt_time = time.time()
                conn.execute(text(sql_drop_rpt_dt))
                conn.execute(text("COMMIT"))
                print('-----Finished dropping ' + str(t) + ' table(s)-----')
                print('Runtime: ', time.time() - drop_rpt_dt_time)
            except:
                print("Unable to drop table(s)")
                conn.close()
                sys.exit()
    print('DONE!!!')
    end_time = time.time()
    #print('Job finished at: ', end_time)
    print('Total runtime: ', end_time - start_time  )
    #close files and connection
    create_tbl.close()
    insert.close()
    drop.close()
    conn.close()


In [9]:
job_feature('30-06-2023')

--------STARTING DROPING TABLE(S)--------
drop table POS_MERCHANT_AMT_6M
Unable to drop table(s)


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
import time
start_time = time.time()
print('Running time: ', round(time.time() - start_time),2)